In [ ]:
from pynq import Overlay
import numpy as np

In [ ]:
ol = Overlay("dSB_8J_1024_axi.bit")

In [ ]:
ol.ip_dict.keys()

In [ ]:
dSB = ol.dSB_0

In [ ]:
dSB.register_map

In [ ]:
N = 1024

In [7]:
A = (-1)*np.ones((N, N)) + np.eye(N)

In [8]:
A

array([[ 0., -1., -1., ..., -1., -1., -1.],
       [-1.,  0., -1., ..., -1., -1., -1.],
       [-1., -1.,  0., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ...,  0., -1., -1.],
       [-1., -1., -1., ..., -1.,  0., -1.],
       [-1., -1., -1., ..., -1., -1.,  0.]])

In [9]:
J = A

In [10]:
#sum_J_power = 0
#for i in range(N):
#    for j in range(N):
#        sum_J_power += J[i][j] * J[i][j]

In [11]:
#xi = 0.5 * np.sqrt(N - 1) / np.sqrt(sum_J_power)

In [12]:
#xi

In [13]:
def ap_fixed_to_int(binary_str):
    """
    Convert a 16-bit two's complement fixed-point number to an integer.
    
    Args:
    binary_str (str): A 16-bit binary string
    
    Returns:
    float: The decimal integer value
    """
    # Validate input
    if len(binary_str) != 16:
        raise ValueError("Input must be a 16-bit binary string")
    
    # Check if it's a negative number (first bit is 1)
    if binary_str[0] == '1':
        # For negative numbers, invert the bits and add 1
        inverted = ''.join('1' if bit == '0' else '0' for bit in binary_str)
        return -(int(inverted, 2) + 1)
    else:
        # For positive numbers, simply convert to decimal
        return int(binary_str, 2)

def detailed_ap_fixed_conversion(binary_str):
    """
    Provide detailed conversion of a 16-bit two's complement fixed-point number.
    
    Args:
    binary_str (str): A 16-bit binary string
    
    Returns:
    dict: Detailed conversion information
    """
    # Validate input
    if len(binary_str) != 16:
        raise ValueError("Input must be a 16-bit binary string")
    
    # Breakdown the bits
    sign_bit = binary_str[0]
    integer_bits = binary_str[1:4]  # 3 magnitude bits for integer part
    fractional_bits = binary_str[4:]  # 12 fractional bits
    
    # Convert to integer value
    if sign_bit == '1':
        # Negative number: invert and add 1
        inverted = ''.join('1' if bit == '0' else '0' for bit in binary_str)
        int_value = -(int(inverted, 2) + 1)
    else:
        int_value = int(binary_str, 2)
    
    return {
        'binary': binary_str,
        'sign_bit': sign_bit,
        'integer_bits': integer_bits,
        'fractional_bits': fractional_bits,
        'integer_value': int_value,
        'decimal_representation': int_value / (2 ** 12)  # Divide by 2^12 due to 12 fractional bits
    }



In [14]:
def get_result_x(r_x):
    count = 0
    for j in range(N):
        count = j;
        c_a = count%2
        if c_a == 1:
            tmpt = dSB.read(0x800+4*int(count/2))
            cyc = bin(tmpt)[2:34]
            #print(cyc)
            cyc = cyc.zfill(32)
            #print(cyc)
            a = ap_fixed_to_int(cyc[0:16])
            #print(len(cyc[18:34]))
            #print(cyc)
            #print(cyc[18:34])
            b = ap_fixed_to_int(cyc[16:32])
            #print(a>>12, b>>12)
            #r_x[j-1] = np.sign(a)#>>12
            #r_x[j] = np.sign(b)#>>12
            r_x[j-1] = a/(2**14)
            r_x[j] = b/(2**14)
rx = np.zeros(N)

In [15]:
from pynq import allocate

In [16]:
# 分配内存 - 例如为4096×4096位分配空间
array_size = N * N  # 以uint32为单位的大小
data_buffer = allocate(shape=(array_size,), dtype=np.uint8)
data_buffer_out = allocate(shape=(array_size,), dtype=np.uint8)

In [17]:
# 设置特定位的值的函数
def set_bit(data, row, col, value):
    bit_position = row * N + col
    byte_index = bit_position // 8
    bit_offset = bit_position % 8
    
    if value:
        data[bit_position] = np.uint8(1)<<6
        #data[byte_index] |= (1 << bit_offset)  # 设置位为1
    else:
        data[bit_position] = np.uint8(0)<<6
        #data[byte_index] &= ~(1 << bit_offset)  # 设置位为0

# 填充数据（示例）
# 例如，创建一个棋盘图案
for i in range(N):
    for j in range(N):
        # 创建棋盘格图案作为示例
        value = 1 if i!=j else 0
        set_bit(data_buffer, i, j, value)

In [18]:
# 获取分配内存的设备地址
device_address = data_buffer.device_address

# 将64位地址拆分为两个32位部分
address_lower = device_address & 0xFFFFFFFF         # 低32位
address_upper = (device_address >> 32) & 0xFFFFFFFF # 高32位

# 写入地址到寄存器
dSB.write(0x010, address_lower)  # 低32位地址写入0x010寄存器
dSB.write(0x014, address_upper)  # 高32位地址写入0x014寄存器

In [19]:
# 获取分配内存的设备地址
device_address_out = data_buffer_out.device_address

# 将64位地址拆分为两个32位部分
address_lower_out = device_address_out & 0xFFFFFFFF         # 低32位
address_upper_out = (device_address_out >> 32) & 0xFFFFFFFF # 高32位

# 写入地址到寄存器
dSB.write(0x01c, address_lower_out)  # 低32位地址写入0x010寄存器
dSB.write(0x020, address_upper_out)  # 高32位地址写入0x014寄存器

In [20]:
x = 0.02 * (np.random.rand(N) - 0.5)
y = 0.02 * (np.random.rand(N) - 0.5)
for i in range(N):
    x[i] *= (2**14)
    y[i] *= (2**14)
x=x.astype(np.int16)
y=y.astype(np.int16)

In [21]:
%%time
dt = 0.002*(2**14)
print(dt)
xi = 0.015625*(2**14)
print(xi)
tmpt = np.zeros((2),dtype="int16")
tmpt[0] = np.int16(xi)
dSB.write(0x028, bytes(tmpt))
tmpt[0] = np.int16(dt)
dSB.write(0x030, bytes(tmpt))



count = 0
for j in range(N):
    count = j;
    c_a = count%2
    if c_a == 0:
        tmpt = np.zeros((2),dtype="int16")
    tmpt[c_a] = x[j]
    if c_a == 1:
        val = bytes(tmpt)
        dSB.write(0x800+4*int(count/2), val)

count = 0
for j in range(N):
    count = j;
    c_a = count%2
    if c_a == 0:
        tmpt = np.zeros((2),dtype="int16")
    tmpt[c_a] = y[j]
    if c_a == 1:
        val = bytes(tmpt)
        dSB.write(0x1000+4*int(count/2), val)



32.768
256.0
CPU times: user 84.7 ms, sys: 3 µs, total: 84.7 ms
Wall time: 84.5 ms


In [22]:
%%time
dSB.write(0,1)

finished = dSB.read(0)
while(finished!=14):
    finished = dSB.read(0)

CPU times: user 277 ms, sys: 0 ns, total: 277 ms
Wall time: 275 ms


In [23]:
dSB.read(0)

4

In [24]:
get_result_x(rx)
np.sum(rx<0)

512

In [25]:
rx

array([ 0.03790283,  0.03936768,  0.05981445, ...,  0.03204346,
        0.04223633, -0.1907959 ])

In [26]:
import numpy as np

def get_bit_matrix(data, N):
    """
    从数据缓冲区读取位信息，并构建N×N的矩阵
    
    参数:
        data: 包含位数据的缓冲区
        N: 矩阵的维度
    
    返回:
        N×N的numpy矩阵，包含位值（0或1）
    """
    # 创建一个N×N的零矩阵
    matrix = np.zeros((N, N), dtype=np.uint8)
    
    # 遍历矩阵的每个位置
    for i in range(N):
        for j in range(N):
            bit_position = i * N + j
            # 根据你的设置函数逻辑，检查位值
            if data[bit_position] == np.uint8(1)<<6:
                matrix[i, j] = 1
            else:
                matrix[i, j] = 0
    
    return matrix

# 使用示例
# 假设 data_buffer 和 N 已经定义
# matrix = get_bit_matrix(data_buffer, N)
# print(matrix)

In [27]:
get_bit_matrix(data_buffer, N)

array([[0, 1, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 1, 1],
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0]], dtype=uint8)